In [1]:
# %pip install numpy
# %pip install pandas
# %pip install torch

In [2]:
#installing the required libraries
# %pip install transformers
# %pip install torchtext
# %pip install sentencepiece
# %pip install datasets
# %pip install torchmetrics
# %pip install matplotlib
# %pip install seaborn
# %pip install scikit-learn
# #huggiface download
# %pip install huggingface_hub


In [3]:
# %pip install llama-cpp-python

In [4]:
from huggingface_hub import hf_hub_download

/home/biomedialab/anaconda3/envs/viveksdmlenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_cpp import Llama

In [6]:
#download the model llama 2 13b chat gguf
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # the model is in gguf format

In [7]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#print toal path
print(model_path)

/home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf


In [8]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )
     

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rop

llama_model_loader: - kv  13:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  14:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  17:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  18:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   81 tensors
llama_model_loader: - type q5_K:  281 tensors
llama_model_loader: - type q6_K:    1 tensors
llm_load_vocab: special tokens definition check successful ( 259/32000 ).
llm_load_print_meta: format           = GGUF V2
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = SPM
llm_load

In [15]:
prompt= f"is 22 prime?"

prompt_template=f'''You are a math assistant. I will ask you to find wheather given number is prime or not. Please answer in the correct format. For example, if I ask 'is 5 prime?' , you should answer 'Yes' if 5 is a prime number or 'No' if 5 is not a prime number. Do not include any other information in your answer.
USER: {prompt}

ASSISTANT:'''
        

In [16]:
response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.88 ms /     2 runs   (    0.44 ms per token,  2262.44 tokens per second)
llama_print_timings: prompt eval time =    2702.45 ms /    55 tokens (   49.14 ms per token,    20.35 tokens per second)
llama_print_timings:        eval time =     677.83 ms /     1 runs   (  677.83 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =    3386.91 ms /    56 tokens


In [18]:
print(response["choices"][0]["text"])

 No


In [19]:
import json

In [22]:
#load file in data folder int_addition.json
with open('primality_test.json') as f:
    data = json.load(f)

In [23]:
#data as pandas dataframe
import pandas as pd
df = pd.DataFrame(data)
#last 5 rows
df

,primes,composites
0,3187,99160
1,59509,12005
2,58787,33309
3,22861,73881
4,89519,78681
...,...,...
4995,38053,83445
4996,74797,35077
4997,78823,18386
4998,61751,19887


In [25]:
df.iloc[1, 0]

59509

In [26]:
import time





for i in range(0, len(df)):
    
    if i%100==0:
        print(f"Completed {i} rows")
        
    try:
        
        prompt= f"Is {df.iloc[i, 0]} prime?"
        
        prompt_template=f'''You are a math assistant. I will ask you to find wheather given number is prime or not. Please answer in the correct format. For example, if I ask 'Is 5 prime?' , you should answer 'Yes' if 5 is a prime number or 'No' if 5 is not a prime number. Do not include any other information in your answer.

        USER: {prompt}
        
        ASSISTANT:'''
        
        response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=False)
        
        with open('prime_test_prime.txt', 'a') as f:
            answer=response["choices"][0]["text"]
            f.write(f"{df.iloc[i, 0]} = {answer} \n")
            
            
    except:
        
        with open('prime_test_prime.txt', 'a') as f:
            f.write(f"{df.iloc[i, 0]} = Error \n")
            
        time.sleep(2)

Completed 0 rows


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.93 ms /     2 runs   (    0.47 ms per token,  2150.54 tokens per second)
llama_print_timings: prompt eval time =    3161.75 ms /    65 tokens (   48.64 ms per token,    20.56 tokens per second)
llama_print_timings:        eval time =     676.72 ms /     1 runs   (  676.72 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =    3844.77 ms /    66 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.87 ms /     2 runs   (    0.44 ms per token,  2293.58 tokens per second)
llama_print_timings: prompt eval time =     812.12 ms /    16 tokens (   50.76 ms per token,    19.70 tokens per second)
llama_print_timings:        eval time =     711.51 ms /     1 runs   (  711.51 ms per token,     1.41 tokens per second)
llama_print_timings:       to

Completed 100 rows



llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.79 ms /     2 runs   (    0.40 ms per token,  2522.07 tokens per second)
llama_print_timings: prompt eval time =     818.50 ms /    16 tokens (   51.16 ms per token,    19.55 tokens per second)
llama_print_timings:        eval time =     655.59 ms /     1 runs   (  655.59 ms per token,     1.53 tokens per second)
llama_print_timings:       total time =    1479.28 ms /    17 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.83 ms /     2 runs   (    0.41 ms per token,  2421.31 tokens per second)
llama_print_timings: prompt eval time =     835.65 ms /    16 tokens (   52.23 ms per token,    19.15 tokens per second)
llama_print_timings:        eval time =     652.45 ms /     1 runs   (  652.45 ms per token,     1.53 tokens per second)
llama_print_timings:       total time =    1493.81 ms /    17 

Completed 200 rows


...


llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.84 ms /     2 runs   (    0.42 ms per token,  2378.12 tokens per second)
llama_print_timings: prompt eval time =     777.27 ms /    15 tokens (   51.82 ms per token,    19.30 tokens per second)
llama_print_timings:        eval time =     670.09 ms /     1 runs   (  670.09 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =    1453.03 ms /    16 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.87 ms /     2 runs   (    0.43 ms per token,  2301.50 tokens per second)
llama_print_timings: prompt eval time =     812.15 ms /    15 tokens (   54.14 ms per token,    18.47 tokens per second)
llama_print_timings:        eval time =     681.21 ms /     1 runs   (  681.21 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =    1498.98 ms /    16 

Completed 4800 rows



llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.90 ms /     2 runs   (    0.45 ms per token,  2229.65 tokens per second)
llama_print_timings: prompt eval time =     810.68 ms /    15 tokens (   54.05 ms per token,    18.50 tokens per second)
llama_print_timings:        eval time =     659.98 ms /     1 runs   (  659.98 ms per token,     1.52 tokens per second)
llama_print_timings:       total time =    1476.91 ms /    16 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.81 ms /     2 runs   (    0.40 ms per token,  2481.39 tokens per second)
llama_print_timings: prompt eval time =     884.99 ms /    16 tokens (   55.31 ms per token,    18.08 tokens per second)
llama_print_timings:        eval time =     658.37 ms /     1 runs   (  658.37 ms per token,     1.52 tokens per second)
llama_print_timings:       total time =    1549.07 ms /    17 

Completed 4900 rows



llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.78 ms /     2 runs   (    0.39 ms per token,  2551.02 tokens per second)
llama_print_timings: prompt eval time =     837.10 ms /    16 tokens (   52.32 ms per token,    19.11 tokens per second)
llama_print_timings:        eval time =     676.92 ms /     1 runs   (  676.92 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =    1519.27 ms /    17 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3512.73 ms
llama_print_timings:      sample time =       0.88 ms /     2 runs   (    0.44 ms per token,  2275.31 tokens per second)
llama_print_timings: prompt eval time =     796.93 ms /    15 tokens (   53.13 ms per token,    18.82 tokens per second)
llama_print_timings:        eval time =     660.73 ms /     1 runs   (  660.73 ms per token,     1.51 tokens per second)
llama_print_timings:       total time =    1463.29 ms /    16 